In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
import geocoder
import tensorflow_hub as hub

In [ ]:
train=pd.read_csv(r"C:\Data Science for Telecom\NLP\Twitter Prediction\train.csv")

In [ ]:
train.head()

In [ ]:
test=pd.read_csv(r"C:\Data Science for Telecom\NLP\Twitter Prediction\test.csv")

In [ ]:
test.head()

In [ ]:
print('There are {} rows and {} columns in train'.format(train.shape[0],train.shape[1]))
print('There are {} rows and {} columns in train'.format(test.shape[0],test.shape[1]))

In [ ]:
# Class distribution

x=train.target.value_counts()
sns.barplot(x.index,x)
plt.gca().set_ylabel('samples')

## Character length distribution

In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,5))
tweet_len_prob =train[train['target']==1]['text'].str.len()
ax1.hist(tweet_len_prob,color='red')
ax1.set_title("Disaster_tweets")
tweet_len_noprob=train[train['target']==0]['text'].str.len()
ax2.hist(tweet_len_noprob,color='green')
ax2.set_title("Non_disastor_tweets")

## Word Length Distribution

In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,5))
tweet_len_word_dis = train[train['target']==1]['text'].str.split().map(lambda x:len(x))
ax1.hist(tweet_len_word_dis,color='red')
ax1.set_title("Disastor_tweets")
tweet_len_word_nodis = train[train['target']==0]['text'].str.split().map(lambda x:len(x))
ax2.hist(tweet_len_word_nodis,color='green')
ax2.set_title("No_Disastor_tweets")

## Perc distribution of word length per tweet

In [ ]:
fig,(ax1,ax2)= plt.subplots(1,2,figsize=(10,5))
word=train[train['target']==1]['text'].str.split().apply(lambda x:[len(i) for i in x])
sns.distplot(word.map(lambda x:np.mean(x)),ax=ax1,color='red')
ax1.set_title("disastor_tweet")
word_nodis =train[train['target']==0]['text'].str.split().apply(lambda x:[len(i) for i in x])
sns.distplot(word.map(lambda x:np.mean(x)),ax=ax2,color='green')
ax2.set_title("no_disastor_tweet")

In [ ]:
def create_corpus(target):
    corpus=[]
    for x in train[train['target']==target]['text'].str.split():
        for i in x:
            corpus.append(i)
    return(corpus)

In [ ]:
corpus=create_corpus(0)
len(corpus)

In [ ]:
dic=defaultdict(int)
for word in corpus:
    if word in stop:
        dic[word]+=1
dic

In [ ]:
top=sorted(dic.items(),key=lambda x:x[1],reverse=True)[:10]
top

In [ ]:
x,y=zip(*top)
plt.bar(x,y)

In [ ]:
corpus1=create_corpus(1)
dic1=defaultdict(int)
for word in corpus1:
    if word in stop:
        dic1[word]+=1


In [ ]:
top1 = sorted(dic1.items(),key=lambda x:x[1],reverse=True)[:10]

In [ ]:
top1

In [ ]:
x1,y1=zip(*top1)
plt.bar(x1,y1)

## Spread of Punctuations

In [ ]:
plt.figure(figsize=(10,5))
corpus=create_corpus(1)
dic=defaultdict(int)
import string
special=string.punctuation
for i in corpus:
    if i in special:
        dic[i]+=1
x,y=zip(*dic.items())
plt.bar(x,y)

In [ ]:
corpus=create_corpus(0)
dic=defaultdict(int)
for i in corpus:
    if i in special:
        dic[i]+=1
x,y=zip(*dic.items())
plt.bar(x,y)

In [ ]:
counter=Counter(corpus)
most=counter.most_common()
x=[]
y=[]
for word,count in most[:40]:
    if word not in stop:
        x.append(word)
        y.append(count)

In [ ]:
sns.barplot(x=x,y=y)

In [ ]:
df=train.append(test).reset_index(drop=True)

In [ ]:
example="New competition launched :https://www.kaggle.com/c/nlp-getting-started"

In [ ]:
def remove_url(text):
    url = re.compile(r'https://\S+|www\.\S+')
    return url.sub(r'',text)
remove_url(example)

In [ ]:
df['text']=df['text'].apply(lambda x : remove_url(x))

In [ ]:
def remove_html(text):
    html=re.compile('<.*?>')
    return html.sub(r'',text)


In [ ]:
example = """<div>
<h1>Real or Fake</h1>
<p>Kaggle </p>
<a href="https://www.kaggle.com/c/nlp-getting-started">getting started</a>
</div>"""

print(remove_html(example))

In [ ]:
df['text']=df['text'].apply(lambda x : remove_html(x))

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [ ]:
remove_emoji("Omg #another Earthquake 😔😔")

In [ ]:
df['text']=df['text'].apply(lambda x: remove_emoji(x))

In [ ]:
def remove_punct(text):
    try:
        table =str.maketrans("","",string.punctuation)
        return text.translate(table)
    except:
        return(text)

In [ ]:
print(remove_punct("is , an the value"))

In [ ]:
df['text']=df['text'].apply(lambda x: remove_punct(x))

In [ ]:
from spellchecker import SpellChecker

In [ ]:
spell=SpellChecker()
def spell_correction(text):
    try:
        spelled =[]
        uncorrect =spell.unknown(text.split())
        for word in text.split():
            if word in uncorrect:
                spelled.append(spell.correction(word))
            else:
                spelled.append(word)
        return(" ".join(spelled))
    except:
        return(text)

In [ ]:
example="kiss me plese"
spell_correction(example)

In [ ]:
def ascii_remover(text):
    try:
        all_ascii = ''.join(char for char in text if ord(char) < 128)
        return(all_ascii)
    except:
        return(text)

In [ ]:
 ascii_remover("45Ã¥Â¡ 5'12.53N   14Ã¥Â¡ 7'24.93E")


In [ ]:
df['text']=df['text'].apply(lambda x:ascii_remover(x))

In [ ]:
df['location']=df['location'].apply(lambda x:ascii_remover(x))

In [ ]:
def replace(text):
    try:
        
        text_replace=text.replace("T: ", "")
        return(text_replace)
    except:
        return(text)

In [ ]:
df['location']=df['location'].apply(lambda x:replace(x))

In [ ]:
#df['location']=df['location'].apply(lambda x: remove_punct(x))

In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
def reverse_geocoder(text):
    
    geolocator = Nominatim()
    try:
        location = geolocator.reverse(text)
        address=location.address
        if address is None:
            return(text)
        else:
            return(address)
    except:
        return(text)
    

In [ ]:
df1=df[df['id']==9662]
df1

In [ ]:
df1['location']=df1['location'].apply(lambda x:reverse_geocoder(x))
df1

In [ ]:
df['location']=df['location'].apply(lambda x:reverse_geocoder(x))

In [ ]:
df['location']=df['location'].apply(lambda x:spell_correction(x))

In [ ]:
def string_capitalize(text):
    try:
        words=text.split()
        word1=[]
        for word in words:
            word1.append(word.capitalize())
        return(" ".join(word1))
        
    except:
        return(text)

In [ ]:
df['location']=df['location'].apply(lambda x:string_capitalize(x))
df['text']=df['text'].apply(lambda x:string_capitalize(x))
df['text']=df['text'].apply(lambda x:spell_correction(x))

In [ ]:
from geotext import GeoText

In [ ]:
def geo_cities(text):
    try:
        places=GeoText(text)
        city=places.cities
        country=places.countries

        if len(city)>0:
            
            return(city[0])
        elif len(country)>0:
            return(country[0])
        else :
            return(text)
    except:
        return(text)

In [ ]:
df['location']=df['location'].apply(lambda x: geo_cities(x))

In [ ]:
df['location']=df['location'].apply(lambda x: remove_punct(x))

In [ ]:
df['keyword']=df['keyword'].apply(lambda x:spell_correction(x))

In [ ]:
#nltk.download('averaged_perceptron_tagger')

In [ ]:
from pattern.text.en import singularize
def singular(text):
    try:
        words=text.split()
        word1=[]
        for word in words:
            word1.append(singularize(word))
        return(" ".join(word1))
        
    except:
        return(text)
        

In [ ]:
df['text']=df['text'].apply(lambda x:singular(x))

In [ ]:
def string_lower(text):
    try:
        words=text.split()
        word1=[]
        for word in words:
            word1.append(word.lower())
        return(" ".join(word1))
        
    except:
        return(text)

In [ ]:
df['text']=df['text'].apply(lambda x:string_lower(x))

In [ ]:

def keyword(text):
    disastor=['flooding','wildfire','bombed','bagging','kill','dead','apocalypse','calamity','catastrophe','collapse','crash','debacle','defeat','emergency','failure','fiasco','flood','harm','hazard','holocaust','mishap','setback','tragedy','woe','adversity','affliction','bale','bane','blight','blow','bust','casualty','cataclysm','collision','depression','exigency','fall','flop','grief','misadventure','mischance','misfortune','reverse','rock','rough','ruin','ruination','slip','stroke','undoing','upset','washout','act of God','bad luck','bad news','fell stroke','hard luck','hot water','ill luck','the worst','crack-up','disaster','fender-bender','fluke','pileup','rear ender','smash','smashup','stack-up','total','wrack-up','bad break','bummer','can of worms','clutch','contretemps','crunch','difficulty','distress','downer','drag','evil eye','hard knocks','hard times','hardship','hurting','ill fortune','jam','jinx','kiss of death','misery','on the skids','pain in the neck','poison','sorrow','suffering','tough luck','trial','trouble','annoyance','bad trip','disappointment','irritation','bum trip','depressing experience','raw deal','rotten hand','unhappy situation','unpleasant experience','unpleasant situation','unpleasent','burden','bête noir','curse','despair','destruction','downfall','fatal attraction','nuisance','pest','plague','scourge','torment','venom','balk','bolt from the blue','bombshell','chagrin','comedown','disgruntlement','frustration','jolt','letdown','shock','chance','contingency','accident','alluvion','culmination','curtains','denouement','desolation','devastation','end','fatality','finale','havoc','ill','infliction','meltdown','termination','upshot','waterloo','wreck','embarrassment','predicament','agitation','clamor','commotion','ferment','furor','outcry','quaking','rocking','seism','shaking','tottering','trembling','tumult','turbulence','upheaval','upturn','big trouble','change','climacteric','climax','confrontation','corner','crossroad','crux','deadlock','dilemma','dire straits','entanglement','extremity','height','hot potato','hour of decision','imbroglio','impasse','juncture','mess','moment of truth','necessity','pass','perplexity','pickle','pinch','plight','point of no return','pressure','puzzle','quandary','situation','stew','strait','trauma','turning point','urgency','Cancer','cross','evil','hydra','ordeal','pestilence','tribulation','vexation','voodoo','beating','blue ruin','breakdown','defeasance','dissolution','drubbing','licking','overthrow','reversal','rout','shellacking','trouncing','vanquishment','bitter pill','blind alley','blunder','bringdown','discouragement','dud','error','false alarm','faux pas','fizzle','flash in the pan','inefficacy','lemon','miscalculation','mistake','obstacle','old one-two','destitution','hard time','holy mess','indigence','need','poverty','privation','rigor','rotten luck','scrape','straits','throe','ticklish spot','tough break','unholy mess','vicissitude','want','Judgment Day','Moira','annihilation','circumstance','conclusion','condemnation','death','decree','destination','destiny','fixed future','foreordination','fortune','handwriting on wall','judgment','karma','kismet','lap of the gods','lot','opinion','portion','predestination','predetermination','sentence','verdict','way the ball bounces','way the cookie crumbles','acme','acuteness','apex','apogee','border','bound','boundary','brim','brink','butt','consummation','crisis','depth','edge','excess','extreme','extremes','frontier','last','margin','maximum','nadir','outside','pinnacle','pole','remote','rim','terminal','terminus','tip','top','verge','vertex','zenith','abasement','capitulation','degradation','diminution','dive','drop','humiliation','loss','resignation','surrender','tumble','deadliness','destructiveness','dying','inevitability','lethality','lethalness','mortality','necrosis','noxiousness','poisonousness','virulence','abortion','botched situation','dumb thing to do','dumb trick','farce','flap','miscarriage','route','screwup','stunt','bomb','loser','nonstarter','Herculean task','asperity','austerity','case','danger','discomfort','drudgery','fatigue','grievance','injury','labor','oppression','peril','persecution','rainy day','toil','travail','uphill battle','worry','ache','black and blue','boo-boo','bruise','chop','detriment','disadvantage','disservice','down','gash','ill-treatment','mark','mischief','nick','ouch','outrage','pain','pang','prejudice','scratch','sore','soreness','wound','wrong','bereavement','cost','damage','debit','debt','deficiency','depletion','deprivation','disappearance','dispossession','forfeiture','hurt','impairment','losing','mislaying','misplacing','perdition','retardation','sacrifice','shrinkage','squandering','waste','wreckage','lapse','ill-fortune','rear-ender','crime','crying shame','regret','shame','sin','clobbering','confusion','flight','hiding','retreat','romp','shambles','shutout','thrashing','trashing','walkover','waxing','whipping','about-face','alteration','convulsion','disorder','disruption','disturbance','eruption','explosion','flip-flop','new ball-game','new deal','outbreak','outburst','revolution','shakeout','stirring','switch','temblor','tremor','turmoil','turnaround','punishment','retribution','downpour','erosion','gully','agony','anguish','bemoaning','blues','care','dejection','deploring','dole','gloom','grieving','headache','heartache','heartbreak','lamentation','melancholy','rain','rue','sadness','unhappiness','wretchedness','attack','armageddon','aftershock','typhoon','asteroid','tsunami','natural disasters','volcano','tornado','avalanche','earthquake','blizzard','drought','bushfire','dust storm','magma','twister','windstorm','heat wave','cyclone','forest fire','fire','hailstorm','lava','lightning','high-pressure','hail','hurricane','seismic','whirlpool','Richter scale','whirlwind','cloud','thunderstorm','barometer','gale','blackout','gust','force','low-pressure','volt','snowstorm','rainstorm','storm','nimbus','violent storm','sandstorm','Beaufort scale','fatal','cumulonimbus','lost','money','tension','uproot','underground','destroy','arsonist','wind scale','arson','rescue','permafrost','fault','shelter','ablaze'
]
    try:
        words=text.split()
        word1=[]
        for word in words:
            if word in disastor:
                word1.append(word)
        return(",".join(word1))
    
    except:
        return("")

In [ ]:
y1 = df.loc[df['keyword']=='']
index_y1=list(y1.index.values)
print(index_y1)

for i in index_y1:
    df['keyword'].iat[i]=keyword(df['text'].iat[i])

In [ ]:
df['keyword'].iat[3]

In [ ]:
df['text']=df['text'].apply(lambda x:string_capitalize(x))

To Capture those cities/geographical areas which are made up of 2 words with a space between them e.g. South Tampa , North Wales etc..

In [ ]:
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import Tree
def get_continuous_chunks(text, label):
    try:
        chunked = ne_chunk(pos_tag(word_tokenize(text)))
        prev = None
        continuous_chunk = []
        current_chunk = []

        for subtree in chunked:
            if type(subtree) == Tree and subtree.label() == label:
                current_chunk.append(" ".join([token for token, pos in subtree.leaves()]))
            elif current_chunk:
                named_entity = " ".join(current_chunk)
                if named_entity not in continuous_chunk:
                    continuous_chunk.append(named_entity)
                    current_chunk = []
            else:
                continue

        return continuous_chunk
    except:
        return("")

In [ ]:
get_continuous_chunks("Haha South Tampa Is Getting Flooded Hah Wait A Second I Live In South Tampa What Am I Gonna Do What Am I Gonna Do Fvck Flooding",'GPE')

In [ ]:
y = df.loc[df['location']==""]
index_y=list(y.index.values)
print(index_y)

for i in index_y:
    df['location'].iat[i]=get_continuous_chunks(df['text'].iat[i],'LOCATION')

In [ ]:
y = df.loc[df['location']==""]
index_y=list(y.index.values)
print(index_y)

for i in index_y:
    df['location'].iat[i]=get_continuous_chunks(df['text'].iat[i],'GPE')

In [ ]:
df.head(100)

In [ ]:
def remove_ing(text):
    try:
        words=text.split()
        word1=[]
        for word in words:
            word1.append(word.replace("ing",""))
        return(" ".join(word1))
    
    except:
        return("")

In [ ]:
df['text']=df['text'].apply(lambda x:remove_ing(x))
y1 = df.loc[df['keyword']=='']
index_y1=list(y1.index.values)
print(index_y1)

for i in index_y1:
    df['keyword'].iat[i]=keyword(df['text'].iat[i])

In [ ]:
df.to_csv(r'C:\Data Science for Telecom\NLP\Twitter Prediction\output_datafile.csv', index=False)